In [94]:
%run "./1. Data Loading.ipynb"

X: (800, 360, 25)
Y: (800,)


In [95]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import StratifiedKFold, train_test_split

In [96]:
print(set(regions[1]))

{'Somatomotor', 'Auditory', 'Language', 'Frontopariet', 'Default', 'Posterior-Mu', 'Visual2', 'Orbito-Affec', 'Cingulo-Oper', 'Ventral-Mult', 'Dorsal-atten', 'Visual1'}


In [97]:
# Helper function:
def find_indices_matching_string(arr, target_string):
    indices = []
    for index, item in enumerate(arr):
        if item == target_string:
            indices.append(index)
    return indices

In [98]:
# Average related conditions in each run.
averaged_data = {}
for subject_index, subject_id in enumerate(manifest):
    averaged_data[subject_id] = []

    for run_index, run in enumerate(manifest[subject_id]):

        neut_block = []
        fear_block = []

        for block_index, block_lookup in enumerate(run["condition_spans"]): 

            if (block_index+1 == 6):
                break
            
            start_index = block_lookup['Frames'][0]
            end_index = block_lookup['Frames'][1]
            condition = 0 if block_lookup['Condition'] == "Neut" else 1
            
            block_data = run["data"][:, start_index : end_index]

            if (condition == 0):
                neut_block.append(block_data)
            elif (condition == 1):
                fear_block.append(block_data)

        neut = np.mean(neut_block, axis=0)
        fear = np.mean(fear_block, axis=0)

        averaged_data[subject_id].append({
            "neut": neut,
            "fear": fear
        })

print(averaged_data["100307"][0]["neut"].shape)
print(averaged_data["100307"][0]["fear"].shape)

(360, 25)
(360, 25)


In [99]:
# Generate correlation matrix for our seed region against a target network.
seed_region = "L_V1"
seed_index = list(regions[0]).index(seed_region)

target_network = "Default"
target_network_indices = find_indices_matching_string(regions[1], target_network)

# NOTE: seed_region must not be in target_region or logic error. 
if regions[1][seed_index] == target_network:
    print("Error: seed_region must not be in target_region.")
    quit()

for subject_index, subject_id in enumerate(averaged_data):
    for run_index, run in enumerate(averaged_data[subject_id]):
        neut_data = run["neut"]
        fear_data = run["fear"]

        neut_correlation_matrix = np.corrcoef(neut_data)
        fear_correlation_matrix = np.corrcoef(fear_data)

        neut_seed_matrix = neut_correlation_matrix[seed_index, target_network_indices]
        fear_seed_matrix = fear_correlation_matrix[seed_index, target_network_indices]

        averaged_data[subject_id][run_index]["neut_seed_matrix"] = neut_seed_matrix
        averaged_data[subject_id][run_index]["fear_seed_matrix"] = fear_seed_matrix

In [100]:
# Prepare the data for modeling:
X = []
Y = []

for subject_index, subject_id in enumerate(averaged_data):
    for run_index, run in enumerate(averaged_data[subject_id]):

            X.append(averaged_data[subject_id][run_index]["neut_seed_matrix"])
            Y.append(0)

            X.append(averaged_data[subject_id][run_index]["neut_seed_matrix"])
            Y.append(1)

X = np.array(X)
Y = np.array(Y)

print(f"X: {X.shape}")
print(f"Y: {Y.shape}")

X: (400, 23)
Y: (400,)


In [101]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [102]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, train_size=0.8, test_size=0.20)

model_dir = "models"

num_samples = 400
num_features = 23
num_classes = 2
hidden_size = 50

num_epochs = 10
num_folds = 5

learning_rate = 0.001
batch_size = 25

k_sampler = StratifiedKFold(n_splits=num_folds, shuffle=True)

# Create a SummaryWriter object
writer = SummaryWriter()

for fold_index, (train_indices, test_indices) in enumerate(k_sampler.split(X_train, Y_train)):
    print(f"Fold {fold_index + 1}:")

    np.save(f'./models/model_{fold_index + 1}_train.npy', train_indices)
    np.save(f'./models/model_{fold_index + 1}_test.npy', test_indices)

    train_length = len(train_indices)
    test_length = len(test_indices)
    x_train, y_train = X[train_indices], Y[train_indices]
    x_test, y_test = X[test_indices], Y[test_indices]

    model = SimpleNN(num_features, hidden_size, num_classes).float()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Model Training:
    for epoch_index in range(num_epochs):
        print(f"\tEpoch {epoch_index}:")
        train_loss = 0.0

        model.train()
        for batch_index in range(0, train_length, batch_size):
            inputs = torch.from_numpy(x_train[batch_index: batch_index + batch_size]).float()
            labels = torch.from_numpy(y_train[batch_index: batch_index + batch_size]).long()

            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / (train_length // batch_size)
        print(f"\t\tTrain Loss: {avg_train_loss:.4f}")

        # Write training loss to TensorBoard
        writer.add_scalar('Loss/Train', avg_train_loss, fold_index * num_epochs + epoch_index)

        val_loss = 0.0
        correct = 0
        total = 0

        # Model Evaluation:
        model.eval()
        with torch.no_grad():
            inputs = torch.from_numpy(x_test).float()
            labels = torch.from_numpy(y_test).long()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Compute accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Print validation loss and accuracy
        print(f"\t\tValidation Loss: {val_loss / test_length:.4f}")
        print(f"\t\tValidation Accuracy: {(100 * correct / total):.2f}%")

        # Write validation loss and accuracy to TensorBoard
        writer.add_scalar('Loss/Validation', val_loss / test_length, fold_index * num_epochs + epoch_index)
        writer.add_scalar('Accuracy/Validation', 100 * correct / total, fold_index * num_epochs + epoch_index)
    
    # Save the model.
    torch.save(model, f'./models/model_{fold_index + 1}.pth')

# Close the SummaryWriter
writer.close()


Fold 1:
	Epoch 0:
		Train Loss: 0.7757
		Validation Loss: 0.0108
		Validation Accuracy: 51.56%
	Epoch 1:
		Train Loss: 0.7676
		Validation Loss: 0.0109
		Validation Accuracy: 51.56%
	Epoch 2:
		Train Loss: 0.7647
		Validation Loss: 0.0110
		Validation Accuracy: 48.44%
	Epoch 3:
		Train Loss: 0.7634
		Validation Loss: 0.0110
		Validation Accuracy: 46.88%
	Epoch 4:
		Train Loss: 0.7621
		Validation Loss: 0.0110
		Validation Accuracy: 46.88%
	Epoch 5:
		Train Loss: 0.7610
		Validation Loss: 0.0111
		Validation Accuracy: 39.06%
	Epoch 6:
		Train Loss: 0.7600
		Validation Loss: 0.0111
		Validation Accuracy: 37.50%
	Epoch 7:
		Train Loss: 0.7589
		Validation Loss: 0.0112
		Validation Accuracy: 35.94%
	Epoch 8:
		Train Loss: 0.7579
		Validation Loss: 0.0112
		Validation Accuracy: 34.38%
	Epoch 9:
		Train Loss: 0.7569
		Validation Loss: 0.0113
		Validation Accuracy: 28.12%
Fold 2:
	Epoch 0:
		Train Loss: 0.7638
		Validation Loss: 0.0109
		Validation Accuracy: 43.75%
	Epoch 1:
		Train Loss: 0.7